In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import xgboost as xgb
import lightgbm as lgb
import holidays
from datetime import date
import re
%matplotlib inline

In [0]:
train = pd.read_csv('/content/drive/My Drive/AV LTFS Finhack2/train.csv', parse_dates=['application_date'])
test = pd.read_csv('/content/drive/My Drive/AV LTFS Finhack2/test.csv', parse_dates=['application_date'])
car = pd.read_csv('/content/drive/My Drive/AV LTFS Finhack2/CarSales.csv')

In [0]:
train_sum = train.groupby(by=['application_date', 'segment'])['case_count'].sum().reset_index().rename(columns={"case_count": "case_count_sum"})
train = train.merge(train_sum, on=['application_date', 'segment'], how='left')
train.shape

(80402, 7)

In [0]:
holidays = holidays.India(years=[2017, 2018, 2019])
holidays.append('2017-03-28')
holidays.append('2017-09-30')
holidays.append('2017-10-19')
holidays.append('2018-03-02')
holidays.append('2018-03-18')
holidays.append('2018-09-13')
holidays.append('2018-10-19')
holidays.append('2018-11-07')
holidays.append('2019-03-21')
holidays.append('2019-04-06')
holidays.append('2019-09-02')
holidays.append('2019-10-08')
holidays.append('2019-10-27')

In [0]:
train['is_holiday'] = train['application_date'].dt.date.isin(holidays)

In [0]:
def add_datepart(df, fldname, drop=True, time=False):
    "Helper function that adds columns relevant to a date."
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)

In [0]:
train.drop_duplicates(['application_date', 'segment'], inplace=True)
train.shape

(1650, 8)

In [0]:
add_datepart(train, 'application_date')
# add_datepart(train_2, 'application_date')

In [0]:
train['application_Is_month_end'] = train['application_Is_month_end'].astype(int)
train['application_Is_month_start'] = train['application_Is_month_start'].astype(int)
train['application_Is_quarter_end'] = train['application_Is_quarter_end'].astype(int)
train['application_Is_quarter_start'] = train['application_Is_quarter_start'].astype(int)
train['application_Is_year_end'] = train['application_Is_year_end'].astype(int)
train['application_Is_year_start'] = train['application_Is_year_start'].astype(int)

In [0]:
train.drop(['branch_id', 'state', 'zone'], axis=1, inplace=True)

In [0]:
train.drop(['case_count'], axis=1, inplace=True)
train.rename(columns={'case_count_sum': 'case_count'}, inplace=True)

In [0]:
train['case_count'] = np.log1p(train['case_count'])

In [0]:
X_train, X_valid, y_train, y_valid = train_test_split(train.drop(['case_count'], axis=1), train['case_count'], 
                                                    test_size=0.2, random_state=423)

In [0]:
lightgbm_params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'num_leaves': 50,
    'metric': 'mape',
    'max_depth': -1, 
    'learning_rate': 0.1
}
n_estimators = 3000

In [0]:
d_train = lgb.Dataset(X_train, label=y_train)
d_valid = lgb.Dataset(X_valid, label=y_valid)
watchlist = [d_valid]

lightgbm_model = lgb.train(lightgbm_params, d_train, n_estimators, watchlist, verbose_eval=50, 
                           early_stopping_rounds=50)

Training until validation scores don't improve for 50 rounds.
[50]	valid_0's mape: 0.040961
[100]	valid_0's mape: 0.0417909
Early stopping, best iteration is:
[52]	valid_0's mape: 0.040888


In [0]:
test['is_holiday'] = test['application_date'].dt.date.isin(holidays)

In [0]:
add_datepart(test, 'application_date')

In [0]:
test['application_Is_month_end'] = test['application_Is_month_end'].astype(int)
test['application_Is_month_start'] = test['application_Is_month_start'].astype(int)
test['application_Is_quarter_end'] = test['application_Is_quarter_end'].astype(int)
test['application_Is_quarter_start'] = test['application_Is_quarter_start'].astype(int)
test['application_Is_year_end'] = test['application_Is_year_end'].astype(int)
test['application_Is_year_start'] = test['application_Is_year_start'].astype(int)

In [0]:
preds = lightgbm_model.predict(test.drop(['id'], axis=1), num_iteration=lightgbm_model.best_iteration)

In [0]:
preds = np.expm1(preds)

In [0]:
test_sub = pd.read_csv('/content/drive/My Drive/AV LTFS Finhack2/test.csv', parse_dates=['application_date'])

In [0]:
sub = pd.DataFrame()
sub['id'] = test_sub['id']
sub['application_date'] = test_sub['application_date']
sub['segment'] = test_sub['segment']
sub['case_count'] = preds
sub.head()

,id,application_date,segment,case_count
0,1,2019-07-06,1,3199.342486
1,2,2019-07-07,1,1356.919164
2,3,2019-07-08,1,4229.451808
3,4,2019-07-09,1,3386.738862
4,5,2019-07-10,1,4025.102113


In [0]:
sub.to_csv('lgb_cars.csv', index=False, header=True)